## Import code

In [22]:
from src.Data import * # Class containing the data
from src.Assignment import * # Class containing an assignment
from src.Model import * # Class containing a Pulp model used for optimization
from src.ModelFracStrong import * # Class containing a Pulp model for finding an fractionally strongly stable stochastic improvement
from src.DataGen import * # Generate student preferences and school priorities
from src.DA_STB import * # Generate DA assignment with single tie-breaking (STB)

## Run code generated data
Specify the number of students and schools, and run the models for this data.

In [16]:
# Generate random data
parameters = DataGenParam(mean_pref = 6, capacity_ratio = 1) # Default parameters, except for mean_pref and capacity_ratio
MyData = generate_data(n_students=6, n_schools=6, parameters = parameters, name="Test_DataGen", print_data=False, seed = 15)

In [17]:
# Print data if desired
print(MyData)

The data instance has the following properties: 

	6 students.
	6 schools. 

 	PREFERENCES:
	0	2 5 3 4 1 0 
	1	3 4 1 5 2 0 
	2	1 3 4 2 5 0 
	3	1 3 4 2 5 0 
	4	3 1 2 5 4 0 
	5	4 3 1 2 5 0 


 	CAPACITIES & PRIORITIES:
	0	1	{5 2} {1 3} {0 4} 
	1	1	{1 3} {4 0} {2 5} 
	2	1	{3 5} {1 2} {4 0} 
	3	1	{5 2} {3 4} {1 0} 
	4	2	{2 4} {3 5} {1 0} 
	5	2	{3 1} {4 2} {0 5} 



In [18]:
# Generate the assignment from DA with Single Tie-Breaking with n_iter samples
n_iter = 1000
A = DA_STB(MyData, n_iter, 0, True)
print(A.assignment)

Students in ties: 6
Tie-breaking rules needed: 720
Tie-breaking rules sampled: 720


100%|██████████| 720/720 [00:00<00:00, 3320.06it/s]

[[0.  0.  0.5 0.  0.  0.5]
 [0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  1.  0.  0. ]
 [0.  1.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.  0.  0.5]
 [0.  0.  0.  0.  1.  0. ]]


In [20]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
    # 'STABLE' refers to maximizing the fraction of STABLE matchings in the decomposition
MyModel = Model(MyData, A, True)
q = MyModel.Solve("IMPR_RANK", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)


Average rank before optimization: 1.8333333333333333.


Set parameter Username
Set parameter LicenseID to value 2595965
Academic license - for non-commercial use only - expires 2025-12-05
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5845 rows, 2737 columns and 22177 nonzeros
Model fingerprint: 0x77399837
Variable types: 1405 continuous, 1332 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 2e+00]
Presolve removed 4461 rows and 2137 columns
Presolve time: 0.04s
Presolved: 1384 rows, 600 columns, 3675 nonzeros
Variable types: 489 continuous, 111 integer (111 binary)
Found heuristic solution: objective 1.8333333

Root relaxation: object

In [ ]:
# Print the solution
MyModel.print_solution()

In [ ]:
# Asses and visualize the difference
diff = Assignment(MyData, q.assignment - A.assignment, "TestDataGen_Diff")
diff.visualize()

## Heuristic Fractional strongly stable random matching

In [23]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
MyModelFS = ModelFracStrong(MyData, A, True)
q = MyModelFS.Solve("IMPR_RANK", "GUROBI", True)
#q = MyModel.Solve("STABLE", "GUROBI", True)

AttributeError: 'ModelFracStrong' object has no attribute 'M'

## Run code manual data
Manually enter data.

In [3]:
# Define preferences of the students
# 'pref[i][k]' contains the position of the k-th ranked school in the preferences.
# We assume the preferences to be strict
# Note that preferences can be strict. We indicate this by a tuple () in the list.

# Example paper
n_stud = 4
n_schools = 4

file_name = "Ex_paper"

# Preferences students
pref = [['1', '3', '4', '2'],
       ['1','4','3','2'],
       # ['1', '4'],
       ['2','3', '4', '1'],
       ['2', '4', '3', '1']]

# Priorities schools
prior = [[('A', 'B'), 'C', 'D'],
        [('C', 'D'), 'A', 'B'],
        ['B', 'D', ('A', 'C')],
        ['A', 'C', ('B', 'D')]]


# Capacities schools
cap = [1,1,1,1]

# Names of students and schools
ID_stud = ["A", "B", "C", "D"]
ID_school = ["1", "2", "3", "4"]

# Also create the random matching upon which we want to improve
p = np.zeros(shape=(n_stud, n_schools))
p[0][0] = 1/2
p[1][0] = 1/2
p[2][1] = 1/2
p[3][1] = 1/2
p[0][2] = 3/8
p[2][2] = 3/8
p[1][3] = 3/8
p[3][3] = 3/8
p[0][3] = 1/8
p[2][3] = 1/8
p[1][2] = 1/8
p[3][2] = 1/8

In [ ]:
# Generate a data instance (and print it)
MyData = Data(n_stud, n_schools, pref, prior, cap, ID_stud, ID_school, file_name)
print(MyData)

In [ ]:
# Generate an Assignment instance (and visualize it)
A = Assignment(MyData, p, "Ex_paper")

# To visualize assignment
A.visualize()

In [ ]:
# Solve the formulations
    # 'IMPR_RANK' refers to minimizing the expected rank while ensuring ex-post stability
    # 'STABLE' refers to maximizing the fraction of STABLE matchings in the decomposition
MyModel = Model(MyData, A, False)
q = MyModel.Solve("IMPR_RANK", "GUROBI", False)
#q = MyModel.Solve("STABLE", "GUROBI", True)

In [ ]:
# Print the solution
MyModel.print_solution()

In [ ]:
# Asses and visualize the difference
diff = Assignment(MyData, q.assignment - p, "Ex_paper_Diff")
diff.visualize()